In [1]:
## Pasos:

#1. RandomForest - MDI
#2. Seleccionar mejores caractéristicas.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston
from sklearn.inspection import permutation_importance
from matplotlib import pyplot as plt
from sklearn.base import clone
plt.style.use('ggplot')
pd.set_option("display.max_columns", 50)

%matplotlib inline

### 1. Leer datos pre-procesados

In [3]:
#Lectura del dataset

X_TRAIN_DF = pd.read_csv('data/02_data_intermedia/X_MI_entrenamiento_preprocesado.csv', header=0, sep=",")
X_TEST_DF  = pd.read_csv('data/02_data_intermedia/X_MI_pruebas_preprocesado.csv', header=0, sep=",")
Y_TRAIN_DF = pd.read_csv('data/02_data_intermedia/y_MI_entrenamiento_preprocesado.csv', header=0, sep=",")
Y_TEST_DF  = pd.read_csv('data/02_data_intermedia/y_MI_pruebas_preprocesado.csv', header=0, sep=",")

In [4]:
var_objetivo_lst = [
    'FIBR_PREDS','PREDS_TAH','JELUD_TAH','FIBR_JELUD',
    'A_V_BLOK','OTEK_LANC','RAZRIV','DRESSLER','ZSN',
    'REC_IM','P_IM_STEN'
]

### 2. Selección de mejores características usando MDI y RandomForest

In [5]:
X_TRAIN_CARACTERIZADO_DF = pd.DataFrame()
X_TEST_CARACTERIZADO_DF  = pd.DataFrame()
for var_objetivo in var_objetivo_lst:
    
    X_train = X_TRAIN_DF.loc[X_TRAIN_DF["var_objetivo"] == var_objetivo]
    X_test  = X_TEST_DF.loc[X_TEST_DF["var_objetivo"] == var_objetivo] 
    y_train = Y_TRAIN_DF.loc[Y_TRAIN_DF["var_objetivo"]== var_objetivo ,[var_objetivo]]
    
    X_train.drop(["var_objetivo"], axis=1, inplace=True)
    rf = RandomForestRegressor(80, n_jobs=-1, oob_score=True)
    rf.fit(X_train, y_train)

    print(rf.feature_importances_)

    importances = pd.DataFrame(
        {"columns":X_train.columns, 
         "importances":rf.feature_importances_}).sort_values("importances", ascending=False)

    print(importances.style.bar())

    columnas_importantes = importances.loc[importances.importances>=0.0009,"columns"].unique()
    X_train_caracterizado = X_train[columnas_importantes]
    X_test_caracterizado = X_test[columnas_importantes]
    X_train_caracterizado["var_objetivo"] = var_objetivo
    X_test_caracterizado["var_objetivo"]  = var_objetivo
    
    X_TRAIN_CARACTERIZADO_DF = pd.concat([X_TRAIN_CARACTERIZADO_DF,X_train_caracterizado], axis=0)
    X_TEST_CARACTERIZADO_DF  = pd.concat([X_TEST_CARACTERIZADO_DF,X_test_caracterizado], axis=0)


    print(f"El proceso de selección de características para la variable objetivo {var_objetivo} obtiene \
            un total de {len(columnas_importantes)} características.")

D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-5-d3de15a69c67>:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


[6.99021448e-02 4.31817201e-02 2.25669817e-02 9.92709877e-03
 1.05621635e-02 1.03830768e-01 1.76195324e-02 1.30656336e-03
 2.09932615e-02 5.83542957e-03 7.27492437e-03 1.76066448e-03
 3.93142260e-04 1.26228112e-02 1.14281627e-03 5.16663748e-03
 1.43513774e-04 5.36809668e-05 5.96891499e-05 1.17280930e-03
 0.00000000e+00 3.67180355e-05 5.38935394e-05 1.65444568e-05
 8.24718221e-03 2.06382792e-03 3.56553071e-04 2.29424559e-03
 7.88878715e-03 4.47323910e-03 5.62971155e-04 2.93381383e-03
 1.68733672e-02 2.35734433e-02 2.87771398e-03 3.96360405e-04
 3.46659021e-03 2.79177317e-03 6.74652566e-05 6.29359543e-04
 1.23674231e-02 3.65880061e-02 1.15092419e-02 8.32866817e-03
 6.74150062e-03 9.23746688e-03 1.94019032e-03 1.41203728e-03
 3.99999119e-04 1.20856896e-02 9.59509832e-03 2.05885712e-02
 3.16895456e-03 6.03237212e-03 4.96290127e-03 9.32725696e-02
 1.37927670e-03 3.44150761e-03 3.10031613e-03 4.18819606e-04
 1.96766531e-05 3.42600043e-03 9.18460327e-05 1.06271498e-03
 1.89985215e-02 4.289171

<ipython-input-5-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-5-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo
D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

[7.91101887e-04 4.48554343e-03 7.21425242e-03 9.58415577e-04
 1.92223117e-03 1.06283612e-02 1.00449218e-01 5.56303534e-04
 3.63995303e-03 1.06019396e-02 5.03901476e-04 1.85044151e-05
 0.00000000e+00 2.86033903e-04 1.84456457e-05 0.00000000e+00
 7.02701393e-05 0.00000000e+00 3.57692489e-05 0.00000000e+00
 0.00000000e+00 1.83213094e-04 4.50811902e-05 3.59543875e-05
 1.52082158e-02 2.32571923e-03 1.83310161e-05 1.84368685e-03
 8.86865274e-03 9.60578562e-04 3.61030810e-05 9.08284201e-05
 9.28678043e-03 5.57372756e-02 3.03041428e-03 2.04726275e-04
 3.77654160e-04 8.97225898e-02 9.89114449e-05 1.36929843e-04
 5.76266703e-03 2.81162681e-03 2.42339328e-03 1.14803317e-02
 1.39588017e-03 3.01484429e-02 6.42339862e-03 9.87029949e-04
 7.53326866e-04 4.56258916e-03 3.49484428e-03 2.85369841e-03
 3.26266153e-03 2.14958224e-03 3.58871853e-03 2.93260312e-03
 4.45719766e-03 3.84108730e-02 7.18508825e-03 6.21026181e-03
 3.12080580e-02 5.75103757e-03 1.72763608e-02 7.84157207e-03
 2.86546807e-03 2.367811

<ipython-input-5-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-5-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo
D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

...

<ipython-input-5-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-5-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo
D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

[1.18917828e-02 4.02453032e-02 2.37100310e-02 4.30267424e-02
 4.10988673e-02 3.45976075e-02 1.67050899e-02 1.55334949e-03
 1.07127362e-02 4.83664101e-03 1.26282160e-03 1.37356013e-03
 1.43445650e-03 3.75556953e-03 4.49348570e-03 4.94576809e-05
 2.23078525e-04 2.73458531e-03 5.04410514e-04 1.68452342e-03
 2.00523884e-04 2.67041235e-04 1.23520592e-04 1.50556435e-03
 3.22943938e-03 3.08517865e-03 1.54628548e-03 2.10414010e-03
 8.64310147e-03 3.43115868e-03 1.03116789e-05 1.82771010e-03
 1.58641386e-02 2.01416687e-02 6.55639365e-03 7.56565638e-04
 1.04877431e-03 2.82084986e-04 9.55918325e-04 2.57921635e-03
 1.31007557e-02 3.23066309e-02 1.42004267e-02 1.00549065e-02
 2.62846220e-03 3.91566486e-03 1.41187968e-03 8.61160481e-04
 1.46312584e-03 4.15725416e-03 7.22818278e-04 1.90302999e-03
 1.81880964e-03 7.42287651e-03 6.33105846e-03 1.01343306e-03
 1.39481653e-02 2.83513308e-03 1.45070614e-02 2.20879186e-03
 2.43388523e-03 7.33744911e-04 1.45043190e-03 4.86045542e-03
 1.32832611e-03 2.584670

<ipython-input-5-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-5-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo
D:\Users\coliverac\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

[6.93388208e-03 2.25552375e-02 2.00788326e-02 3.54354069e-02
 1.05403815e-01 3.54141577e-02 9.68383765e-03 1.52074584e-03
 1.16390816e-02 8.91987730e-03 2.13847082e-03 3.59411103e-05
 4.12633352e-04 2.29616584e-03 1.97261992e-03 2.71825367e-05
 1.62409877e-04 0.00000000e+00 5.02104927e-04 3.39829492e-03
 0.00000000e+00 9.42226026e-05 3.83858631e-05 6.81244627e-04
 4.57837331e-03 1.22708990e-03 7.49961509e-04 4.97834754e-03
 1.49825216e-02 9.46426576e-04 5.50288584e-05 2.12248182e-03
 3.55325247e-02 2.82004689e-02 5.16864885e-03 2.94994372e-04
 2.53369253e-03 3.28318814e-04 2.14647180e-04 4.69760505e-04
 1.50640368e-01 2.04516573e-02 4.50909388e-02 8.63834247e-03
 2.99201993e-03 5.14507952e-03 2.01015846e-03 2.45387855e-03
 1.77238984e-03 4.20142745e-03 2.87534722e-03 6.25564321e-03
 2.55131969e-03 7.97807607e-03 4.50799390e-03 1.80714209e-03
 8.00301189e-04 4.12549342e-04 3.02582005e-04 8.57199744e-04
 4.65344782e-04 1.60410441e-03 2.15115914e-03 5.45414070e-04
 7.36536386e-04 3.554666

<ipython-input-5-d3de15a69c67>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_caracterizado["var_objetivo"] = var_objetivo
<ipython-input-5-d3de15a69c67>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_caracterizado["var_objetivo"]  = var_objetivo


### <u>Nota:</u> 
<mark>En este experimento se considera valores mayores a 0.0009 porque ratios de importancia menores no aportan o 
aportan muy poco al modelo. Adicionalmente, se consideró este ratio de corte con el fin de no perder
información significativa para el modelo. </mark>

### 3. Guardar resultados de la selección de características

In [6]:
X_TRAIN_CARACTERIZADO_DF.to_csv('data/03_entrada_modelo/X_MI_entrenamiento_caracterizado.csv',index=False)
X_TEST_CARACTERIZADO_DF.to_csv('data/03_entrada_modelo/X_MI_pruebas_caracterizado.csv',index=False)

Y_TRAIN_DF.to_csv('data/03_entrada_modelo/y_MI_entrenamiento_caracterizado.csv',index=False)
Y_TEST_DF.to_csv('data/03_entrada_modelo/y_MI_pruebas_caracterizado.csv',index=False)

In [13]:
sum_num = 0
for var_objetivo in var_objetivo_lst:
    num = len(X_TRAIN_CARACTERIZADO_DF.loc[X_TRAIN_CARACTERIZADO_DF["var_objetivo"]==var_objetivo].dropna(axis='columns').columns.tolist())
    
    sum_num = sum_num + num
    print(num)

69
61
59
59
60
65
55
61
73
76
66


In [15]:
sum_num/11

64.0